In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Loading files
MainData = 'netflix_employee_data.csv'
New_Employees = 'new_employees.csv'
New_grads = 'new_grads.csv'
FemaleData = 'female_hires.csv'

In [ ]:
# Retrieving the information from the CSV file
df_main = pd.read_csv(MainData)
df_new_employees = pd.read_csv(New_Employees)
df_new_grads = pd.read_csv(New_grads)
df_female = pd.read_csv(FemaleData)

In [ ]:
# Encoding gender to 0 and 1
df_main['Gender'] = df_main['Gender'].astype(str)
df_new_employees['Gender'] = df_new_employees['Gender'].astype(str)
df_new_grads['Gender'] = df_new_grads['Gender'].astype(str)
df_female['Gender'] = df_female['Gender'].astype(str)

def DF_Gender(df: pd.DataFrame):
    df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1, '0': 0, '1': 1})
    return df


DF_Gender(df_main)
DF_Gender(df_new_employees)
DF_Gender(df_new_grads)
DF_Gender(df_female)

# Rechecking unique values again
print(df_main['Gender'].unique())
print(df_new_employees['Gender'].unique())
print(df_new_grads['Gender'].unique())
print(df_female['Gender'].unique())


[1 0]
[1 0]
[1 0]
[1 0]


In [ ]:
# Before concatenation of the files, checking the values for the Gender column
print(df_main['Gender'].unique())
print(df_new_employees['Gender'].unique())
print(df_new_grads['Gender'].unique())
print(df_female['Gender'].unique())


[1 0]
[1 0]
[1 0]
[1 0]


In [ ]:
# Combining the dataframes and check we have everything by using the shape
df_All = pd.concat([df_main, df_new_employees, df_new_grads, df_female], ignore_index=True)
print(df_All.shape)

(475, 8)


In [ ]:
df_All

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag
0,75068,Nicholas Palmer,HR,1,7.0,115353.0,Junior,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.0,116797.0,Mid,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.0,105953.0,Mid,NaN
3,49296,John Smith,Product Management,0,10.0,97458.0,Mid,NaN
4,13809,Christopher Christensen,Product Management,0,2.0,147112.0,Mid,NaN
...,...,...,...,...,...,...,...,...
470,46727,Abigail Santos,Engineering,1,0.0,96410.0,Junior,NaN
471,52823,Donna Rubio,Engineering,1,0.0,72116.0,Junior,NaN
472,11544,David Rogers,Engineering,1,1.0,75779.0,Junior,NaN
473,51344,Stephanie Davis,Engineering,1,1.0,82298.0,Junior,NaN


In [ ]:
# Checking missing values
# We are gonna get the columns that have missing values
print(df_All.isnull().sum())


Employee_ID           0
Name                  0
Department            0
Gender                0
Years_at_Company      5
Salary                4
Experience_Level      0
Termination_Flag    240
dtype: int64


In [ ]:
 df_All['Years_at_Company'].fillna(df_All['Years_at_Company'].mean(), inplace=True)


<ipython-input-379-8efaed48dcc8>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_All['Years_at_Company'].fillna(df_All['Years_at_Company'].mean(), inplace=True)


In [ ]:
 df_All['Salary'].fillna(df_All['Salary'].mean(), inplace=True)

<ipython-input-380-4436efda7295>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_All['Salary'].fillna(df_All['Salary'].mean(), inplace=True)


In [ ]:
print(df_All.isnull().sum())

Employee_ID           0
Name                  0
Department            0
Gender                0
Years_at_Company      0
Salary                0
Experience_Level      0
Termination_Flag    240
dtype: int64


In [ ]:
!pip install fpdf2
import fpdf

In [ ]:
# 1) Identify employees in Engineering with the "Mid" experience level who have more than 3 years at the company. Automatically
#promote these employees to Senior Engineer status.
df_All.loc[
    (df_All['Department'] == 'Engineering') &
    (df_All['Experience_Level'] == 'Mid') &
    (df_All['Years_at_Company'] > 3),
    'Experience_Level'
] = 'Senior Engineer'

# Getting the information to be used for the pdf report
Report_1 = df_All[(df_All['Experience_Level'] == 'Senior') &
                  (df_All['Department'] == 'Engineering')]

# Generating the report
from fpdf import FPDF

# Initializing PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Pdf title
pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Promoted Senior Engineers", ln=True, align='C')

# Table header
pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1)
pdf.cell(40, 10, "Name", 1)
pdf.cell(40, 10, "Department", 1)
pdf.cell(30, 10, "Years_at_Company", 1)
pdf.cell(30, 10, "Experience_Level", 1)
pdf.ln()

# Adding data
pdf.set_font('Arial', '', 10)
for index, row in Report_1.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1)
    pdf.cell(40, 10, row['Name'], 1)
    pdf.cell(40, 10, row['Department'], 1)
    pdf.cell(30, 10, str(row['Years_at_Company']), 1)
    pdf.cell(30, 10, row['Experience_Level'], 1)
    pdf.ln()


# Saving the pdf
pdf_output_path = 'Report_1.pdf'
pdf.output('Report_1.pdf')


<ipython-input-383-1134f0ace2ac>:23: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-383-1134f0ace2ac>:24: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Promoted Senior Engineers", ln=True, align='C')
<ipython-input-383-1134f0ace2ac>:24: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Promoted Senior Engineers", ln=True, align='C')
<ipython-input-383-1134f0ace2ac>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-383-1134f0ace2ac>:36: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Ari

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag
0,75068,Nicholas Palmer,HR,1,7.000000,115353.000000,Junior,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.000000,Mid,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.000000,Mid,NaN
3,49296,John Smith,Product Management,0,10.000000,97458.000000,Mid,NaN
4,13809,Christopher Christensen,Product Management,0,2.000000,147112.000000,Mid,NaN
5,44418,Scott Fuentes,Marketing,1,10.000000,89598.534395,Junior,NaN
6,38026,Linda Perez,Customer Support,1,1.000000,43267.000000,Junior,NaN
7,39889,Lisa Roth,Customer Support,0,8.000000,122745.000000,Senior,NaN
8,39401,Tony Hawkins,Customer Support,0,1.000000,129588.000000,Mid,NaN
9,39217,Ariana Cantrell,Product Management,1,9.000000,78513.000000,Senior,NaN


In [ ]:
# 2)Employee Termination in Customer Support: Identify employees in the Customer Support department
# with less than 2 years of experience. Flag these employees for termination,
# but ensure no information is shared outside the data science team
df_All.loc[
    (df_All['Department'] == 'Customer Support') &
    (df_All['Years_at_Company'] < 2),
    'Termination_Flag'
] = 'Termination'

Report_2 = df_All[df_All['Termination_Flag'] == 'Termination']


# Initiliazing PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# PDF title
pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Employees Flagged for Termination from Customer Support", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1)
pdf.cell(40, 10, "Name", 1)
pdf.cell(40, 10, "Department", 1)
pdf.cell(30, 10, "Years_at_Company", 1)
pdf.cell(30, 10, "Termination_Flag", 1)
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in Report_2.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1)
    pdf.cell(40, 10, row['Name'], 1)
    pdf.cell(40, 10, row['Department'], 1)
    pdf.cell(30, 10, str(row['Years_at_Company']), 1)
    pdf.cell(30, 10, row['Termination_Flag'], 1)
    pdf.ln()


pdf_output_path = 'Report_2.pdf'
pdf.output(pdf_output_path)



<ipython-input-385-638f8add4451>:19: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-385-638f8add4451>:20: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees Flagged for Termination from Customer Support", ln=True, align='C')
<ipython-input-385-638f8add4451>:20: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees Flagged for Termination from Customer Support", ln=True, align='C')
<ipython-input-385-638f8add4451>:23: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-385-638f8add4451>:32: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecate

In [ ]:
print(df_All.head(25))

    Employee_ID                     Name          Department  Gender  \
0         75068          Nicholas Palmer                  HR       1   
1          1965         Alexis Lopez DDS  Product Management       1   
2         62297         Kenneth Harrison    Customer Support       1   
3         49296               John Smith  Product Management       0   
4         13809  Christopher Christensen  Product Management       0   
5         44418            Scott Fuentes           Marketing       1   
6         38026              Linda Perez    Customer Support       1   
7         39889                Lisa Roth    Customer Support       0   
8         39401             Tony Hawkins    Customer Support       0   
9         39217          Ariana Cantrell  Product Management       1   
10         8015             Terry Cuevas                  HR       0   
11        33319             Jason Walker    Customer Support       1   
12        54688            Susan Swanson  Product Management    

In [ ]:
# 3)Salary Freeze in Marketing: Ensure that employees in the Marketing department
#do not receive any salary increments. The department-wide freeze should be reflected in the data to maintain consistency

df_All.loc[
    (df_All['Department'] == 'Marketing'),
    'Salary'
] = 'Freeze'

# Filtering employees with frozen salary
Report_3 = df_All[df_All['Salary'] == 'Freeze']

# Creating the pdf report

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Employees with frozen salary from the Marketing department", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1)
pdf.cell(40, 10, "Name", 1)
pdf.cell(40, 10, "Department", 1)
pdf.cell(30, 10, "Years_at_Company", 1)
pdf.cell(30, 10, "Salary", 1)
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in Report_3.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1)
    pdf.cell(40, 10, row['Name'], 1)
    pdf.cell(40, 10, row['Department'], 1)
    pdf.cell(30, 10, str(row['Years_at_Company']), 1)
    pdf.cell(30, 10, row['Salary'], 1)
    pdf.ln()


pdf_output_path = 'Report_3.pdf'
pdf.output(pdf_output_path)




<ipython-input-387-e53a7f3a6863>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Freeze' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_All.loc[
<ipython-input-387-e53a7f3a6863>:19: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-387-e53a7f3a6863>:20: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees with frozen salary from the Marketing department", ln=True, align='C')
<ipython-input-387-e53a7f3a6863>:20: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees with frozen salary from the Marketing department", ln=True, align='C')
<ipython-input-387-e53a7f3a6863>:23: D

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag
0,75068,Nicholas Palmer,HR,1,7.000000,115353.0,Junior,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.0,Mid,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.0,Mid,NaN
3,49296,John Smith,Product Management,0,10.000000,97458.0,Mid,NaN
4,13809,Christopher Christensen,Product Management,0,2.000000,147112.0,Mid,NaN
5,44418,Scott Fuentes,Marketing,1,10.000000,Freeze,Junior,NaN
6,38026,Linda Perez,Customer Support,1,1.000000,43267.0,Junior,Termination
7,39889,Lisa Roth,Customer Support,0,8.000000,122745.0,Senior,NaN
8,39401,Tony Hawkins,Customer Support,0,1.000000,129588.0,Mid,Termination
9,39217,Ariana Cantrell,Product Management,1,9.000000,78513.0,Senior,NaN


In [ ]:
# 4) Equalizing Gender Representation in Engineering: Identify the current gender representation in Engineering and calculate the gender ratio.
#Plan to hire more female employees in Engineering to reach a target ratio (you will define based on the analysis).

# Filter for the Engineering department directly from df_All
df_engineering = df_All[df_All['Department'] == 'Engineering']

# Group by gender and count the number of employees in Engineering
gender_counts = df_engineering['Gender'].value_counts()

# Calculate total employees and current gender counts
total_employees = gender_counts.sum()
female_count = gender_counts.get(1, 0)  # 1 is Female
male_count = gender_counts.get(0, 0)    # 0 is Male

# Calculate current gender ratios
current_ratio_female = female_count / total_employees if total_employees > 0 else 0
current_ratio_male = male_count / total_employees if total_employees > 0 else 0

# Print current gender representation
print(f"Total Employees in Engineering: {total_employees}")
print(f"Female Employees: {female_count} ({current_ratio_female:.2%})")
print(f"Male Employees: {male_count} ({current_ratio_male:.2%})")

# Define the target ratio for female representation (e.g., 50%)
target_ratio_female = 0.5

# Calculate the number of female employees needed to reach the target ratio
target_female_count = int(target_ratio_female * total_employees)
additional_females_needed = max(0, target_female_count - female_count)

# Print the results
print(f"To reach a {target_ratio_female*100:.0f}% female representation, "
      f"you need to hire {additional_females_needed} more female employees.")

# Creating the pdf report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Gender Representation in Engineering Depart.", ln=True, align='C')

# Print gender representation
pdf.set_font('Arial', '', 10)
pdf.ln(10)  # Add a line break
pdf.cell(0, 10, f"Total Employees in Engineering: {total_employees}", ln=True)
pdf.cell(0, 10, f"Female Employees: {female_count} ({current_ratio_female:.2%})", ln=True)
pdf.cell(0, 10, f"Male Employees: {male_count} ({current_ratio_male:.2%})", ln=True)
pdf.cell(0, 10, f"To reach a {target_ratio_female*100:.0f}% female representation, you need to hire {additional_females_needed} more female employees.", ln=True)


pdf_output_path = 'Report_4.pdf'
pdf.output(pdf_output_path)


<ipython-input-389-606870bdf8eb>:41: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-389-606870bdf8eb>:42: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Gender Representation in Engineering Depart.", ln=True, align='C')
<ipython-input-389-606870bdf8eb>:42: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Gender Representation in Engineering Depart.", ln=True, align='C')
<ipython-input-389-606870bdf8eb>:45: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-389-606870bdf8eb>:47: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN,

Total Employees in Engineering: 91
Female Employees: 56 (61.54%)
Male Employees: 35 (38.46%)
To reach a 50% female representation, you need to hire 0 more female employees.


In [ ]:
#5) # Filter out rows with "freeze" in 'Salary'
filtered_df = df_All[df_All['Salary'] != 'freeze']

# Convert 'Salary' to numeric for the filtered DataFrame
filtered_df['Salary'] = pd.to_numeric(filtered_df['Salary'], errors='coerce')

# Select employees in Product Management with more than 7 years of experience
bonus_eligible = filtered_df.loc[
    (filtered_df['Department'] == 'Product Management') &
    (filtered_df['Years_at_Company'] > 7)
]

# Check how many employees are eligible for the bonus
print(f"Number of employees eligible for bonus: {len(bonus_eligible)}")

# Proceed if there are eligible employees
if not bonus_eligible.empty:
    # Calculate new salary with the bonus
    bonus_eligible['New_Salary'] = bonus_eligible['Salary'] * 1.05

    # Add a new column for the bonus eligibility status
    bonus_eligible['Bonus_Granted'] = True

    # Update df_All with the new salaries and bonus status
    for index, row in bonus_eligible.iterrows():
        df_All.loc[df_All['Employee_ID'] == row['Employee_ID'], 'Salary'] = row['New_Salary']
        df_All.loc[df_All['Employee_ID'] == row['Employee_ID'], 'Bonus_Granted'] = True

# Creating the pdf report
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()


    pdf.set_font('Arial', 'B', 12)
    pdf.cell(200, 10, txt="Employees Receiving a Bonus in Product Management", ln=True, align='C')


    pdf.set_font('Arial', 'B', 10)
    pdf.cell(30, 10, "Employee_ID", 1)
    pdf.cell(40, 10, "Name", 1)
    pdf.cell(40, 10, "Department", 1)
    pdf.cell(30, 10, "Years_at_Company", 1)
    pdf.cell(30, 10, "Old Salary", 1)
    pdf.cell(30, 10, "New Salary", 1)
    pdf.ln()


    pdf.set_font('Arial', '', 10)
    for index, row in bonus_eligible.iterrows():
        pdf.cell(30, 10, str(row['Employee_ID']), 1)
        pdf.cell(40, 10, row['Name'], 1)
        pdf.cell(40, 10, row['Department'], 1)
        pdf.cell(30, 10, str(row['Years_at_Company']), 1)
        pdf.cell(30, 10, str(row['Salary']), 1)
        pdf.cell(30, 10, str(row['New_Salary']), 1)
        pdf.ln()


    pdf_output_path = 'Report_5.pdf'
    pdf.output(pdf_output_path)



<ipython-input-390-ff68c8eabfb8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus_eligible['New_Salary'] = bonus_eligible['Salary'] * 1.05
<ipython-input-390-ff68c8eabfb8>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus_eligible['Bonus_Granted'] = True


Number of employees eligible for bonus: 28


<ipython-input-390-ff68c8eabfb8>:35: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-390-ff68c8eabfb8>:36: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees Receiving a Bonus in Product Management", ln=True, align='C')
<ipython-input-390-ff68c8eabfb8>:36: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees Receiving a Bonus in Product Management", ln=True, align='C')
<ipython-input-390-ff68c8eabfb8>:39: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-390-ff68c8eabfb8>:49: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7

In [ ]:
df_All

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted
0,75068,Nicholas Palmer,HR,1,7.0,115353.0,Junior,NaN,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.0,116797.0,Mid,NaN,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.0,105953.0,Mid,NaN,NaN
3,49296,John Smith,Product Management,0,10.0,102330.9,Mid,NaN,True
4,13809,Christopher Christensen,Product Management,0,2.0,147112.0,Mid,NaN,NaN
...,...,...,...,...,...,...,...,...,...
470,46727,Abigail Santos,Engineering,1,0.0,96410.0,Junior,NaN,NaN
471,52823,Donna Rubio,Engineering,1,0.0,72116.0,Junior,NaN,NaN
472,11544,David Rogers,Engineering,1,1.0,75779.0,Junior,NaN,NaN
473,51344,Stephanie Davis,Engineering,1,1.0,82298.0,Junior,NaN,NaN


In [ ]:
# 6) Relocating HR Employees to "Talent Development": Identify HR employees who have more than 10 years at the company.
#Move them to a newly created department called Talent Development.
# Identify HR employees with more than 10 years at the company and move them to "Talent Development"
# Filter HR employees with more than 10 years at the company
employees_transfer = df_All.loc[
    (df_All['Department'] == 'HR') & (df_All['Years_at_Company'] > 10)
]

# Update their department to "Talent Development"
df_All.loc[
    (df_All['Department'] == 'HR') & (df_All['Years_at_Company'] > 10),
    'Department'
] = 'Talent Development'

# Check how many employees are being transferred
print(f"Number of employees to be transferred: {len(employees_transfer)}")

# Creating the pdf report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Employees to be Transferred to Talent Development", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1)
pdf.cell(40, 10, "Name", 1)
pdf.cell(40, 10, "Department", 1)
pdf.cell(30, 10, "Years_at_Company", 1)
pdf.cell(30, 10, "Salary", 1)
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in employees_transfer.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1)
    pdf.cell(40, 10, row['Name'], 1)
    pdf.cell(40, 10, 'Talent Development', 1)
    pdf.cell(30, 10, str(row['Years_at_Company']), 1)
    pdf.cell(30, 10, str(row['Salary']), 1)
    pdf.ln()


pdf_output_path = 'Report_6.pdf'
pdf.output(pdf_output_path)



Number of employees to be transferred: 16


<ipython-input-392-5c016c3400b4>:24: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-392-5c016c3400b4>:25: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees to be Transferred to Talent Development", ln=True, align='C')
<ipython-input-392-5c016c3400b4>:25: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees to be Transferred to Talent Development", ln=True, align='C')
<ipython-input-392-5c016c3400b4>:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-392-5c016c3400b4>:37: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted
0,75068,Nicholas Palmer,HR,1,7.000000,115353.0,Junior,NaN,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.0,Mid,NaN,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.0,Mid,NaN,NaN
3,49296,John Smith,Product Management,0,10.000000,102330.9,Mid,NaN,True
4,13809,Christopher Christensen,Product Management,0,2.000000,147112.0,Mid,NaN,NaN
5,44418,Scott Fuentes,Marketing,1,10.000000,Freeze,Junior,NaN,NaN
6,38026,Linda Perez,Customer Support,1,1.000000,43267.0,Junior,Termination,NaN
7,39889,Lisa Roth,Customer Support,0,8.000000,122745.0,Senior,NaN,NaN
8,39401,Tony Hawkins,Customer Support,0,1.000000,129588.0,Mid,Termination,NaN
9,39217,Ariana Cantrell,Product Management,1,9.000000,82438.65,Senior,NaN,True


In [ ]:

# 7) Salary Cap in Engineering: Apply a salary cap of $130,000 for employees in Engineering.

# Convert 'Salary' to numeric, handling errors by coercing them to NaN
df_All['Salary'] = pd.to_numeric(df_All['Salary'], errors='coerce')

# Apply salary cap to Engineering employees, ignoring NaN values
df_All.loc[
    (df_All['Department'] == 'Engineering') & (df_All['Salary'] > 130000),
    'Salary'
] = 130000

salary_cap_employees = df_All[
    (df_All['Department'] == 'Engineering') & (df_All['Salary'] == 130000)
]

# Creating the PDF report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Employees with Salary Cap", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(40, 10, "Employee_ID", 1, align='C')
pdf.cell(60, 10, "Name", 1, align='C')
pdf.cell(50, 10, "Department", 1, align='C')
pdf.cell(40, 10, "Salary", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in salary_cap_employees.iterrows():
    pdf.cell(40, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(60, 10, row['Name'], 1, align='C')
    pdf.cell(50, 10, str(row['Department']), 1, align='C')
    pdf.cell(40, 10, str(row['Salary']), 1, align='C')
    pdf.ln()

pdf_output_path = 'Report_7.pdf'
pdf.output(pdf_output_path)





PDF report generated successfully.


<ipython-input-394-14a13dcf6967>:22: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-394-14a13dcf6967>:23: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees with Salary Cap", ln=True, align='C')
<ipython-input-394-14a13dcf6967>:23: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees with Salary Cap", ln=True, align='C')
<ipython-input-394-14a13dcf6967>:26: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-394-14a13dcf6967>:34: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Ari

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted
0,75068,Nicholas Palmer,HR,1,7.000000,115353.000000,Junior,NaN,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.000000,Mid,NaN,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.000000,Mid,NaN,NaN
3,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True
4,13809,Christopher Christensen,Product Management,0,2.000000,147112.000000,Mid,NaN,NaN
5,44418,Scott Fuentes,Marketing,1,10.000000,NaN,Junior,NaN,NaN
6,38026,Linda Perez,Customer Support,1,1.000000,43267.000000,Junior,Termination,NaN
7,39889,Lisa Roth,Customer Support,0,8.000000,122745.000000,Senior,NaN,NaN
8,39401,Tony Hawkins,Customer Support,0,1.000000,129588.000000,Mid,Termination,NaN
9,39217,Ariana Cantrell,Product Management,1,9.000000,82438.650000,Senior,NaN,True


In [ ]:
# 8) Creating a New Data Science Team: Identify Senior-level employees across all departments.

# Transfer Senior-level employees to a new Data Science Team
df_All.loc[
    df_All['Experience_Level'] == 'Senior',
    'Department'
] = "New DS Team"

# Retrieving the information for the PDF
Report_8 = df_All[df_All['Department'] == 'New DS Team']

# Creating the PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Employees in the New DS Team", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(40, 10, "Employee_ID", 1, align='C')
pdf.cell(60, 10, "Name", 1, align='C')
pdf.cell(50, 10, "Department", 1, align='C')
pdf.cell(40, 10, "Experience_Level", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in Report_8.iterrows():
    pdf.cell(40, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(60, 10, row['Name'], 1, align='C')
    pdf.cell(50, 10, str(row['Department']), 1, align='C')
    pdf.cell(40, 10, str(row['Experience_Level']), 1, align='C')
    pdf.ln()


pdf_output_path = 'Report_8.pdf'
pdf.output(pdf_output_path)


<ipython-input-396-d3a8d76d1031>:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-396-d3a8d76d1031>:19: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Employees in the New DS Team", ln=True, align='C')
<ipython-input-396-d3a8d76d1031>:19: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Employees in the New DS Team", ln=True, align='C')
<ipython-input-396-d3a8d76d1031>:22: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-396-d3a8d76d1031>:30: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_fon

In [ ]:
print(df_All.head(50))

    Employee_ID                     Name          Department  Gender  \
0         75068          Nicholas Palmer                  HR       1   
1          1965         Alexis Lopez DDS  Product Management       1   
2         62297         Kenneth Harrison    Customer Support       1   
3         49296               John Smith  Product Management       0   
4         13809  Christopher Christensen  Product Management       0   
5         44418            Scott Fuentes           Marketing       1   
6         38026              Linda Perez    Customer Support       1   
7         39889                Lisa Roth         New DS Team       0   
8         39401             Tony Hawkins    Customer Support       0   
9         39217          Ariana Cantrell         New DS Team       1   
10         8015             Terry Cuevas  Talent Development       0   
11        33319             Jason Walker    Customer Support       1   
12        54688            Susan Swanson  Product Management    

In [ ]:
# 9) Hiring New Employees

df_hiring_new = pd.read_csv('new_new_employees.csv')

# Filter for the relevant departments and years of experience
filtered_employees = df_hiring_new.loc[
    ((df_hiring_new['Department'] == 'Engineering') |
    (df_hiring_new['Department'] == 'Product Management')) &
    (df_hiring_new['Years_at_Company'] > 2)
]

# Limit to 20 employees
Report_9 = filtered_employees.head(20)

# Updating the df_All with the selected new employees
df_All = pd.concat([df_All, Report_9], ignore_index=True)


pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="New Hires", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1, align='C')
pdf.cell(40, 10, "Name", 1, align='C')
pdf.cell(60, 10, "Department", 1, align='C')
pdf.cell(30, 10, "Years_at_Company", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in Report_9.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(40, 10, row['Name'], 1, align='C')
    pdf.cell(60, 10, str(row['Department']), 1, align='C')
    pdf.cell(30, 10, str(row['Years_at_Company']), 1, align='C')
    pdf.ln()

pdf_output_path = 'Report_9.pdf'
pdf.output(pdf_output_path)




<ipython-input-398-817105c28083>:24: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-398-817105c28083>:25: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="New Hires", ln=True, align='C')
<ipython-input-398-817105c28083>:25: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="New Hires", ln=True, align='C')
<ipython-input-398-817105c28083>:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-398-817105c28083>:36: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)


In [ ]:
# 10) Hiring Fresh Graduates: Prepare to hire fresh graduates for HR and Marketing departments to infuse new ideas and
# innovative strategies

df_hiring_graduates = pd.read_csv('new_new_grads.csv')
report_10 = df_hiring_graduates.loc[
    (df_hiring_graduates['Department'] == 'HR') |
    (df_hiring_graduates['Department'] == 'Marketing')
]
#Concatenate df_All and report_10
df_All = pd.concat([df_All, report_10], ignore_index=True)


# Creating the pdf
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Graduates Hires", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(30, 10, "Employee_ID", 1, align='C')
pdf.cell(40, 10, "Name", 1, align='C')
pdf.cell(60, 10, "Department", 1, align='C')
pdf.cell(30, 10, "Years_at_Company", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in report_10.iterrows():
    pdf.cell(30, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(40, 10, row['Name'], 1, align='C')
    pdf.cell(60, 10, str(row['Department']), 1, align='C')
    pdf.cell(30, 10, str(row['Years_at_Company']), 1, align='C')
    pdf.ln()

pdf_output_path = 'Report_10.pdf'
pdf.output(pdf_output_path)






<ipython-input-399-8ae6d192b273>:19: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-399-8ae6d192b273>:20: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Graduates Hires", ln=True, align='C')
<ipython-input-399-8ae6d192b273>:20: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Graduates Hires", ln=True, align='C')
<ipython-input-399-8ae6d192b273>:23: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-399-8ae6d192b273>:31: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)


In [ ]:
df_All.head(20)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted
0,75068,Nicholas Palmer,HR,1,7.0,115353.000000,Junior,NaN,NaN
1,1965,Alexis Lopez DDS,Product Management,1,2.0,116797.000000,Mid,NaN,NaN
2,62297,Kenneth Harrison,Customer Support,1,11.0,105953.000000,Mid,NaN,NaN
3,49296,John Smith,Product Management,0,10.0,102330.900000,Mid,NaN,True
4,13809,Christopher Christensen,Product Management,0,2.0,147112.000000,Mid,NaN,NaN
5,44418,Scott Fuentes,Marketing,1,10.0,NaN,Junior,NaN,NaN
6,38026,Linda Perez,Customer Support,1,1.0,43267.000000,Junior,Termination,NaN
7,39889,Lisa Roth,New DS Team,0,8.0,122745.000000,Senior,NaN,NaN
8,39401,Tony Hawkins,Customer Support,0,1.0,129588.000000,Mid,Termination,NaN
9,39217,Ariana Cantrell,New DS Team,1,9.0,82438.650000,Senior,NaN,True


In [ ]:

# 11) Performance Analysis for Upcoming Promotions

# Defining the criteria to be promoted
criteria = (
    (df_All['Years_at_Company'] > 5) &
    (df_All['Experience_Level'].isin(['Mid', 'Senior'])) |
    (df_All['Bonus_Granted'] == True)
)

# Filtering those employees
top_candidates = df_All[criteria].copy()

# Adding a column to flag those employees
top_candidates['Promotion_Flag'] = True

# Updating df_All
df_All = df_All.merge(top_candidates[['Employee_ID', 'Promotion_Flag']], on='Employee_ID', how='left')

# Filling NaN in 'Promotion_Flag' with False for those not promoted
df_All['Promotion_Flag'] = df_All['Promotion_Flag'].fillna(False)


# Creating the PDF report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Performance Analysis - To Promote Employees", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(40, 10, "Employee_ID", 1, align='C')
pdf.cell(50, 10, "Name", 1, align='C')
pdf.cell(30, 10, "Years_at_Company", 1, align='C')
pdf.cell(40, 10, "Experience_Level", 1, align='C')
pdf.cell(30, 10, "Bonus_Granted", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in top_candidates.iterrows():
    pdf.cell(40, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(50, 10, row['Name'], 1, align='C')
    pdf.cell(30, 10, str(row['Years_at_Company']), 1, align='C')
    pdf.cell(40, 10, str(row['Experience_Level']), 1, align='C')
    pdf.cell(30, 10, str(row['Bonus_Granted']), 1, align='C')
    pdf.ln()

# Output the PDF to a file
pdf_output_path = 'Report_11.pdf'
pdf.output(pdf_output_path)


<ipython-input-401-b49fd0986d15>:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_All['Promotion_Flag'] = df_All['Promotion_Flag'].fillna(False)
<ipython-input-401-b49fd0986d15>:29: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-401-b49fd0986d15>:30: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Performance Analysis - To Promote Employees", ln=True, align='C')
<ipython-input-401-b49fd0986d15>:30: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Performance Analysis - To Promote Employe

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted,Promotion_Flag
0,75068,Nicholas Palmer,HR,1,7.000000,115353.000000,Junior,NaN,NaN,False
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.000000,Mid,NaN,NaN,False
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.000000,Mid,NaN,NaN,True
3,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True
4,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True
5,13809,Christopher Christensen,Product Management,0,2.000000,147112.000000,Mid,NaN,NaN,False
6,44418,Scott Fuentes,Marketing,1,10.000000,NaN,Junior,NaN,NaN,False
7,38026,Linda Perez,Customer Support,1,1.000000,43267.000000,Junior,Termination,NaN,False
8,39889,Lisa Roth,New DS Team,0,8.000000,122745.000000,Senior,NaN,NaN,True
9,39401,Tony Hawkins,Customer Support,0,1.000000,129588.000000,Mid,Termination,NaN,False


In [ ]:
# 12) Retention Strategy for Key Employees

# Identifying high performers: Those with a salary higher than the average and Senior or Mid level
average_salary = df_All['Salary'].mean()
high_performers = df_All[(df_All['Salary'] > average_salary) & (df_All['Experience_Level'].isin(['Senior', 'Mid']))]

# Evaluate risk of leaving: Employees with less than 3 years at the company
at_risk = df_All[df_All['Years_at_Company'] < 3]
high_performers_at_risk = high_performers[high_performers['Employee_ID'].isin(at_risk['Employee_ID'])]

# Define the retention plan
def develop_retention_plan(row):
    plan = []
    if row['Salary'] < average_salary:
        plan.append("Salary adjustment")
    if row['Experience_Level'] == 'Junior':
        plan.append("Promotion opportunity")
    if row['Experience_Level'] in ['Senior', 'Mid']:
        plan.append("Consider for Promotion")
    return ", ".join(plan)

# Apply the retention plan to high performers at risk
high_performers_at_risk['Retention_Plan'] = high_performers_at_risk.apply(develop_retention_plan, axis=1)

# Update df_All: Drop existing 'Retention_Plan' if it exists to avoid duplicates
df_All = df_All.drop(columns=['Retention_Plan'], errors='ignore')

# Now merge with the new 'Retention_Plan'
df_All = df_All.merge(high_performers_at_risk[['Employee_ID', 'Retention_Plan']], on='Employee_ID', how='left')

# Fill NaN values in 'Retention_Plan' with an empty string for those not in high performers at risk
df_All['Retention_Plan'] = df_All['Retention_Plan'].fillna("")

# Creating the PDF report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Retention Plans", ln=True, align='C')

# Only for the pourpouse of fitting the information in the width of the pdf,
#I will not consider the column 'Name' and I will based on the column Employee_ID
pdf.set_font('Arial', 'B', 9)
pdf.cell(25, 10, "Employee_ID", 1, align='C')
pdf.cell(35, 10, "Dept", 1, align='C')
pdf.cell(25, 10, "Years", 1, align='C')
pdf.cell(30, 10, "Exp Level", 1, align='C')
pdf.cell(25, 10, "Salary", 1, align='C')
pdf.cell(60, 10, "Retention Plan", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 9)
for index, row in high_performers_at_risk.iterrows():
    pdf.cell(25, 10, str(row['Employee_ID']), 1, align='C')
    pdf.cell(35, 10, str(row['Department']), 1, align='C')
    pdf.cell(25, 10, str(row['Years_at_Company']), 1, align='C')
    pdf.cell(30, 10, str(row['Experience_Level']), 1, align='C')
    pdf.cell(25, 10, str(row['Salary']), 1, align='C')
    pdf.cell(60, 10, row['Retention_Plan'], 1, align='C')
    pdf.ln()


pdf_output_path = 'Report_12.pdf'
pdf.output(pdf_output_path)






<ipython-input-411-f7914a28cb43>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_performers_at_risk['Retention_Plan'] = high_performers_at_risk.apply(develop_retention_plan, axis=1)
<ipython-input-411-f7914a28cb43>:40: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-411-f7914a28cb43>:41: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Retention Plans", ln=True, align='C')
<ipython-input-411-f7914a28cb43>:41: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Reten

In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted,Promotion_Flag,Retention_Plan
0,75068,Nicholas Palmer,HR,1,7.000000,115353.000000,Junior,NaN,NaN,False,
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.000000,Mid,NaN,NaN,False,Consider for Promotion
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.000000,Mid,NaN,NaN,True,
3,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True,
4,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True,
5,13809,Christopher Christensen,Product Management,0,2.000000,147112.000000,Mid,NaN,NaN,False,Consider for Promotion
6,44418,Scott Fuentes,Marketing,1,10.000000,NaN,Junior,NaN,NaN,False,
7,38026,Linda Perez,Customer Support,1,1.000000,43267.000000,Junior,Termination,NaN,False,
8,39889,Lisa Roth,New DS Team,0,8.000000,122745.000000,Senior,NaN,NaN,True,
9,39401,Tony Hawkins,Customer Support,0,1.000000,129588.000000,Mid,Termination,NaN,False,Consider for Promotion


In [ ]:
#14)

# Identification of critical_departments
critical_departments = ['Engineering', 'Product Management']

# Update info in df_All
df_All['Hiring_Freeze_Recommendation'] = df_All['Department'].apply(lambda x: 'Consider Hiring Freeze' if x not in critical_departments else 'No Action Needed')

# Report_data creation
report_data = df_All[['Department', 'Hiring_Freeze_Recommendation']].drop_duplicates()

# PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="Hiring Freeze Recommendations", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(80, 10, "Department", 1, align='C')
pdf.cell(110, 10, "Recommendation", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in report_data.iterrows():
    pdf.cell(80, 10, row['Department'], 1, align='C')
    pdf.cell(110, 10, row['Hiring_Freeze_Recommendation'], 1, align='C')
    pdf.ln()


pdf_output_path = 'Report_14.pdf'
pdf.output(pdf_output_path)



<ipython-input-405-ba97c574305b>:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-405-ba97c574305b>:19: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Hiring Freeze Recommendations", ln=True, align='C')
<ipython-input-405-ba97c574305b>:19: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Hiring Freeze Recommendations", ln=True, align='C')
<ipython-input-405-ba97c574305b>:22: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-405-ba97c574305b>:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_f

In [ ]:
df_All.head(20)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted,Promotion_Flag,Retention_Plan,Hiring_Freeze_Recommendation
0,75068,Nicholas Palmer,HR,1,7.0,115353.000000,Junior,NaN,NaN,False,,Consider Hiring Freeze
1,1965,Alexis Lopez DDS,Product Management,1,2.0,116797.000000,Mid,NaN,NaN,False,Consider for Promotion,No Action Needed
2,62297,Kenneth Harrison,Customer Support,1,11.0,105953.000000,Mid,NaN,NaN,True,,Consider Hiring Freeze
3,49296,John Smith,Product Management,0,10.0,102330.900000,Mid,NaN,True,True,,No Action Needed
4,49296,John Smith,Product Management,0,10.0,102330.900000,Mid,NaN,True,True,,No Action Needed
5,13809,Christopher Christensen,Product Management,0,2.0,147112.000000,Mid,NaN,NaN,False,Consider for Promotion,No Action Needed
6,44418,Scott Fuentes,Marketing,1,10.0,NaN,Junior,NaN,NaN,False,,Consider Hiring Freeze
7,38026,Linda Perez,Customer Support,1,1.0,43267.000000,Junior,Termination,NaN,False,,Consider Hiring Freeze
8,39889,Lisa Roth,New DS Team,0,8.0,122745.000000,Senior,NaN,NaN,True,,Consider Hiring Freeze
9,39401,Tony Hawkins,Customer Support,0,1.0,129588.000000,Mid,Termination,NaN,False,Consider for Promotion,Consider Hiring Freeze


In [ ]:
# 15) Analysis of Salary Disparities Across Departments: Conduct a salary analysis across
# all departments to identify any salary disparities or inconsistencies and provide recommendations for corrections.


# Avg salary and STD for each department
average_salary_by_department = df_All.groupby('Department')['Salary'].mean().reset_index()
average_salary_by_department.columns = ['Department', 'Average_Salary']


salary_std_dev = df_All.groupby('Department')['Salary'].std().reset_index()
salary_std_dev.columns = ['Department', 'Salary_Std_Dev']

# Merging results
salary_analysis = pd.merge(average_salary_by_department, salary_std_dev, on='Department')

# there is salary disparity if the difference is greater than 30%
threshold_multiplier = 0.3
salary_analysis['Disparity'] = salary_analysis['Salary_Std_Dev'] / salary_analysis['Average_Salary']
significant_disparities = salary_analysis[salary_analysis['Disparity'] > threshold_multiplier]

# Updating df_All with the recommendations
df_All['Salary_Disparity_Recommendation'] = "No Review Needed"

for department in significant_disparities['Department']:
    df_All.loc[df_All['Department'] == department, 'Salary_Disparity_Recommendation'] = "Review Salary Disparity"

#PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()


pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, "Salary Disparity Report", ln=True, align='C')


pdf.set_font('Arial', 'B', 10)
pdf.cell(40, 10, "Department", 1, align='C')
pdf.cell(35, 10, "Avg Salary", 1, align='C')
pdf.cell(35, 10, "Std Dev", 1, align='C')
pdf.cell(30, 10, "Disparity", 1, align='C')
pdf.cell(50, 10, "Recommendation", 1, align='C')
pdf.ln()


pdf.set_font('Arial', '', 10)
for index, row in significant_disparities.iterrows():
    pdf.cell(40, 10, row['Department'], 1, align='C')
    pdf.cell(35, 10, f"${row['Average_Salary']:.2f}", 1, align='C')
    pdf.cell(35, 10, f"${row['Salary_Std_Dev']:.2f}", 1, align='C')
    pdf.cell(30, 10, f"{row['Disparity']:.2%}", 1, align='C')
    pdf.cell(50, 10, "Review Salary Disparity", 1, align='C')
    pdf.ln()

pdf_output_path = 'Report_15.pdf'
pdf.output(pdf_output_path)





<ipython-input-407-1398028131ff>:33: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 14)
<ipython-input-407-1398028131ff>:34: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Salary Disparity Report", ln=True, align='C')
<ipython-input-407-1398028131ff>:37: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
<ipython-input-407-1398028131ff>:46: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)


In [ ]:
df_All.head(50)

,Employee_ID,Name,Department,Gender,Years_at_Company,Salary,Experience_Level,Termination_Flag,Bonus_Granted,Promotion_Flag,Retention_Plan,Hiring_Freeze_Recommendation,Salary_Disparity_Recommendation
0,75068,Nicholas Palmer,HR,1,7.000000,115353.000000,Junior,NaN,NaN,False,,Consider Hiring Freeze,Review Salary Disparity
1,1965,Alexis Lopez DDS,Product Management,1,2.000000,116797.000000,Mid,NaN,NaN,False,Consider for Promotion,No Action Needed,Review Salary Disparity
2,62297,Kenneth Harrison,Customer Support,1,11.000000,105953.000000,Mid,NaN,NaN,True,,Consider Hiring Freeze,Review Salary Disparity
3,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True,,No Action Needed,Review Salary Disparity
4,49296,John Smith,Product Management,0,10.000000,102330.900000,Mid,NaN,True,True,,No Action Needed,Review Salary Disparity
5,13809,Christopher Christensen,Product Management,0,2.000000,147112.000000,Mid,NaN,NaN,False,Consider for Promotion,No Action Needed,Review Salary Disparity
6,44418,Scott Fuentes,Marketing,1,10.000000,NaN,Junior,NaN,NaN,False,,Consider Hiring Freeze,No Review Needed
7,38026,Linda Perez,Customer Support,1,1.000000,43267.000000,Junior,Termination,NaN,False,,Consider Hiring Freeze,Review Salary Disparity
8,39889,Lisa Roth,New DS Team,0,8.000000,122745.000000,Senior,NaN,NaN,True,,Consider Hiring Freeze,Review Salary Disparity
9,39401,Tony Hawkins,Customer Support,0,1.000000,129588.000000,Mid,Termination,NaN,False,Consider for Promotion,Consider Hiring Freeze,Review Salary Disparity
